In [1]:
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
import io
from googleapiclient.errors import HttpError

In [3]:
scope = ['https://www.googleapis.com/auth/drive']
service_account_file = 'credentials/google_api_credentials.json'
credentials = service_account.Credentials.from_service_account_file(service_account_file, scopes=scope)
service = build('drive', 'v3', credentials=credentials)

In [8]:
# Method to upload file to google drive in a specific folder
def upload_file(file_name, file_path, folder_id):
    file_metadata = {'name': file_name, 'parents': [folder_id]}
    media = MediaFileUpload(file_path, resumable=True)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print('File ID: %s' % file.get('id'))

# Method to fetch folder id from google drive based on folder name
def get_folder_id(folder_name):
    folder_id = None
    page_token = None
    while True:
        response = service.files().list(q="mimeType='application/vnd.google-apps.folder' and name='" + folder_name + "'",
                                              spaces='drive',
                                              fields='nextPageToken, files(id, name)',
                                              pageToken=page_token).execute()
        for file in response.get('files', []):
            folder_id = file.get('id')
        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break
    return folder_id


In [9]:
import docx
doc = docx.Document()
doc.add_paragraph('Hello world!')
doc.save('test.docx')

# Obtain folder id for the folder where you want to upload the file
folder_id = get_folder_id('Job Applications')

# Upload file to google drive
upload_file('test.docx', 'test.docx', folder_id)

File ID: 1VENYBOhVjPgi-pJPz6OsLuew19ZF0MS0
